In [4]:
import pandas as pd
import numpy as np
import glob
import os

In [5]:
def add_rsi_table(inp, window:int=14):
    if isinstance(inp, str):
        df = pd.read_csv(inp)
    elif isinstance(inp, pd.DataFrame):
        df = inp
    else:
        print('please input path to csv or actual dataframe!')
    # Calculate daily price changes
    delta = df['Close'].diff()
    
    # Separate gains and losses
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    
    # Calculate average gain and loss using the rolling window
    avg_gain = gain.rolling(window=window, min_periods=1).mean()
    avg_loss = loss.rolling(window=window, min_periods=1).mean()
    
    # Calculate the Relative Strength (RS)
    rs = avg_gain / avg_loss
    
    # Calculate the RSI
    rsi = 100 - (100 / (1 + rs))
    df['RSI_14'] = rsi

    return df


In [8]:
for path in glob.glob('data/original/*'):
    if os.path.splitext(path)[-1] != '.csv': continue
    df = add_rsi_table(pd.read_csv(path))
    df.to_csv('data/mod/{}'.format(os.path.basename(path)))